In [ ]:
import pandas
import os
import numpy
import plotly.express
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, balanced_accuracy_score, matthews_corrcoef, average_precision_score
from sklearn.tree import plot_tree
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
import xgboost
from sklearn.neighbors import KNeighborsClassifier
import gc
import datetime as dt

In [ ]:
df1 = pandas.read_csv("data/Physical dataset/phy_att_1.csv", encoding="utf-16le", delimiter='\t')
df2 = pandas.read_csv("data/Network datatset/csv/attack_1.csv", encoding="us-ascii", delimiter=',', skipinitialspace=True)


In [ ]:
df1['Time'] = pandas.to_datetime(df1['Time']).dt.round('S')
df2['Time'] = pandas.to_datetime(df2['Time'],format='ISO8601',yearfirst=True).dt.round('S')
df1['Time'] = df1['Time'].dt.strftime('%Y-%d-%m %H:%M:%S')
df2['Time'] = df2['Time'].dt.strftime('%Y-%m-%d %H:%M:%S')


In [ ]:
#df1.drop(['Label','Label_n'], axis=1, inplace=True)
df2.drop(['label','label_n'], axis=1, inplace=True)

In [ ]:
df=pandas.merge(df2,df1, how='outer',on='Time', validate="m:1")


del df1 
del df2

gc.collect()
print(df)


In [ ]:
print(df['Label'].value_counts())

In [ ]:
nb_attaque= df['Label'].value_counts()
print(nb_attaque)
ratio = (nb_attaque.iloc[1]+nb_attaque.iloc[2])/(nb_attaque.iloc[0]+nb_attaque.iloc[1]+nb_attaque.iloc[2])
print(ratio*100)

In [ ]:
colonnes_catégorielles = df.select_dtypes(include=['category', 'object']).columns.tolist()

# Extraction des noms des colonnes numériques
colonnes_numériques = df.select_dtypes(include=['number']).columns.tolist()

print("Colonnes catégorielles :")
print(colonnes_catégorielles)

print("\nColonnes numériques :")
print(colonnes_numériques)

In [ ]:
cat_stats = df[colonnes_catégorielles].describe()
print(cat_stats)
print()
valeur_catégorielles = {}
for col in colonnes_catégorielles:
    valeur_catégorielles[col] = df[col].unique()
    print("%s: %s" % (col, valeur_catégorielles[col]))

In [ ]:
# Extract statistics for each numeric column
numeric_stats = df[colonnes_numériques].describe()

# Print the list of numeric columns
print("Numeric Columns:")
print(colonnes_numériques)

# Print statistics for each numeric column
print("\nStatistics for Numeric Columns:")
print(numeric_stats)

In [ ]:
nan_ratios = df.isna().mean()   
print(nan_ratios)


In [ ]:
target_attack_categories  = df['Label'].unique()
target_attack_categories = target_attack_categories[1:4]
print(target_attack_categories)

In [ ]:
# on récupère tous les labels 1
MM = df[(df['Label_n'] == 1)]
#on récupère toute les colonnes où il y a bien une attaque pour un label 1
matching_rows = (MM['Label'].isin(target_attack_categories))
# on vérifie que l'on a bien que la valeur True
print(matching_rows.all())

In [ ]:
# on récupère tous les labels 0
MM = df[(df['Label_n'] == 0)]
#on récupère toute les colonnes où il y a bien une attaque pour un Label 0
matching_rows =(MM['Label'].isin(["normal"]))
# on vérifie que l'on a bien que la valeur True
print(matching_rows.all())

In [ ]:
fraudulent_entries = df[df['Label_n'] == 1].sample(n=int(100000 * ratio) +1)

# Sample non-fraudulent entries to match the ratio
non_fraudulent_entries = df[df['Label_n'] == 0].sample(n=int(100000 * (1-ratio)))

# Concatenate both sets to create df_reduced
X = pandas.concat([fraudulent_entries, non_fraudulent_entries])
print(X.shape)

del df

gc.collect()

In [ ]:
# Créez un DataFrame exemple

# Remplacez les valeurs '[int]' par 'int' et transformez la colonne en numérique
X['modbus_response'] = X['modbus_response'].str.replace(r'\[(\d+)\]', r'\1', regex=True)
X['modbus_response'] = pandas.to_numeric(X['modbus_response'], errors='coerce')
X['modbus_response'] = X['modbus_response'].fillna(-1)
# La colonne est maintenant de type numérique avec des valeurs numériques
print(X['modbus_response'].unique())
#X = X.fillna(-2)
X = X.dropna()


In [ ]:
nan_ratios = X.isna().mean()   
print(nan_ratios)

In [ ]:
# Store the attack_cat labels in a DataFrame Y as a 1-D array
Y = X['Label_n'].copy()

In [ ]:
print(Y)

In [ ]:
X.drop(['Time','Label','Label_n'], axis=1, inplace=True)

In [ ]:
X = pandas.get_dummies(X)

In [ ]:
colonnes_catégorielles = X.select_dtypes(include=['category', 'object']).columns.tolist()

# Extraction des noms des colonnes numériques
colonnes_numériques = X.select_dtypes(include=['number']).columns.tolist()

print("Colonnes catégorielles :")
print(colonnes_catégorielles)

print("\nColonnes numériques :")
print(colonnes_numériques)

In [ ]:
# Extract train and test data through train_test_split on X and Y
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
Y_pred_proba = clf.predict_proba(X_test)
print(Y_pred)

In [ ]:
print('Confusion matrix:', confusion_matrix(Y_test, Y_pred))
print('Precision:', precision_score(Y_test, Y_pred))
print('Recall:', recall_score(Y_test, Y_pred))
print('Accuracy:', accuracy_score(Y_test, Y_pred))
print('Balanced accuracy:', balanced_accuracy_score(Y_test, Y_pred))
print('Matthews Correlation Coefficient:', matthews_corrcoef(Y_test, Y_pred))
print('AUPRC:', average_precision_score(Y_test, Y_pred))

In [ ]:
# For the XGBClassifier interface in sklearn
# 1. Perform the same analysis
# 2. Plot the relative importance of fields through plot_importance, using importance_type ‘cover’, ‘gain’, ‘weight’
# 3. Show the classifier visually


clf = xgboost.XGBClassifier()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
Y_pred_proba = clf.predict_proba(X_test)

print('Confusion matrix:', confusion_matrix(Y_test, Y_pred))
print('Precision:', precision_score(Y_test, Y_pred))
print('Recall:', recall_score(Y_test, Y_pred))
print('Accuracy:', accuracy_score(Y_test, Y_pred))
print('Balanced accuracy:', balanced_accuracy_score(Y_test, Y_pred))
print('Matthews Correlation Coefficient:', matthews_corrcoef(Y_test, Y_pred))
print('AUPRC:', average_precision_score(Y_test, Y_pred))

fig = plotly.express.imshow(confusion_matrix(Y_test, Y_pred), title='Confusion matrix')
fig.show()

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)

knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
Y_pred_proba = knn.predict_proba(X_test)

print("Confusion matrix:", confusion_matrix(Y_test, Y_pred))
print("Precision:", precision_score(Y_test, Y_pred))
print("Recall:", recall_score(Y_test, Y_pred))
print("Accuracy:", accuracy_score(Y_test, Y_pred))
print("Balanced accuracy:", balanced_accuracy_score(Y_test, Y_pred))
print("Matthews Correlation Coefficient:", matthews_corrcoef(Y_test, Y_pred))
print("AUPRC:", average_precision_score(Y_test, Y_pred))

fig = plotly.express.imshow(confusion_matrix(Y_test, Y_pred), title="Confusion matrix")
fig.show()